In [6]:
import matplotlib.pyplot as plt
from ChernoffFace import ChernoffFace

# Assuming 'df' is your DataFrame and 'column_name' is the column with mixed data
plt.figure(figsize=(10,6))

# Separate numeric and categorical data
numeric_data = df.select_dtypes(include=['float', 'int'])
categorical_data = df.select_dtypes(include=['object'])

# Create a list to store the features for the face
features = []

# Add numeric features to the list
for col in numeric_data.columns:
    features.append(('numeric', col, numeric_data[col].values))

# Add categorical features to the list
for col in categorical_data.columns:
    categories = categorical_data[col].unique()
    features.append(('categorical', col, categories, categorical_data[col].values))

# Create Chernoff Face with mixed data
face = ChernoffFace(features)
plt.title('Chernoff Face for Multivariate Data')
plt.show()

TypeError: 'module' object is not callable

<Figure size 1000x600 with 0 Axes>

In [28]:
import numpy as np
import pandas as pd

In [29]:
df = pd.read_csv("train.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [30]:
df.drop(['PassengerId','Name','Cabin','Ticket'],axis=1, inplace=True)

In [31]:
df['Embarked']=df['Embarked'].fillna(df['Embarked'].mode()[0])

In [32]:
def fill_null_with_random(df, column_name):
    # Count the number of null values in the specified column
    null_count = df[column_name].isnull().sum()

    # Generate random values from the non-null values in the column
    random_values = df[column_name].dropna().sample(null_count, replace=True)

    # Replace null values with random values
    df.loc[df[column_name].isnull(), column_name] = random_values.values

    return df

df = fill_null_with_random(df, "Age")

In [33]:
df.dtypes

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object

In [34]:
def get_dummies_for_categorical(df):

  categorical_cols = df.select_dtypes(include=['object']).columns

  df_encoded = pd.get_dummies(df, columns=categorical_cols, prefix_sep='_')

  for col in df_encoded.columns:
    if df_encoded[col].dtype == 'bool':
      df_encoded[col] = df_encoded[col].astype(int)

  return df_encoded

df_encoded = get_dummies_for_categorical(df)

df_encoded

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.0,0,0,13.0000,0,1,0,0,1
887,1,1,19.0,0,0,30.0000,1,0,0,0,1
888,0,3,40.0,1,2,23.4500,1,0,0,0,1
889,1,1,26.0,0,0,30.0000,0,1,1,0,0


In [36]:
X = df_encoded.drop(['Survived'], axis=1)

In [37]:
y = df_encoded['Survived']

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier  # Replace with your chosen model

# Assuming you have your data loaded into X (features) and y (target variable)

# Split data into training and testing sets (replace test_size with your desired split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features (optional, but often helpful)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a Random Forest model (replace with your chosen model)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Evaluate model performance
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Hyperparameter tuning (optional, but powerful)
from sklearn.model_selection import GridSearchCV

# Define a grid of hyperparameters to search over
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 8]
}

# Create a GridSearchCV object and fit it
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)

# Get the best model with the highest accuracy
best_model = grid_search.best_estimator_

# Evaluate the best model's performance
y_pred_best = best_model.predict(X_test_scaled)
best_accuracy = accuracy_score(y_test, y_pred_best)
print("Best Accuracy:", best_accuracy)

Accuracy: 0.8324022346368715
Best Accuracy: 0.8100558659217877
